## Finetuning Gemma 2B for lease agreement data extraction

### Install dependencies

In [1]:
%pip install pandas numpy peft scikit-learn transformers datasets torch accelerate bitsandbytes huggingface-hub trl -q

Note: you may need to restart the kernel to use updated packages.


### Define the extraction JSON schema

In [2]:
import json

extraction_json_schema = {
  "title": "ExtractedLeaseData",
  "type": "object",
  "properties": {
    "tenant_name": {
      "type": ["string", "null"],
      "description": "The name of the tenant, found in the OCR text."
    },
    "unit_address": {
      "type": ["string", "null"],
      "description": "The unit address found in the OCR text."
    },
    "unit_number": {
      "type": ["string", "null"],
      "description": "The unit number found in the OCR text."
    },
    "unit_type": {
      "type": ["string", "null"],
      "description": "The unit type found in the OCR text."
    },
    "agreement_date": {
      "type": ["string", "null"],
      "format": "date"
    },
    "lease_start": {
      "type": ["string", "null"],
      "format": "date",
      "description": "The date when the lease starts, found in the OCR text."
    },
    "lease_end": {
      "type": ["string", "null"],
      "format": "date",
      "description": "The date when the lease ends, found in the OCR text."
    },
    "lease_auto_renew": {
      "type": ["string", "null"],
      "description": "The type of lease auto renewal, found in the OCR text."
    },
    "hourly_rate": {
      "type": ["number", "null"],
      "description": "The hourly rate found in the OCR text."
    },
    "monthly_rent": {
      "type": ["number", "null"],
      "description": "The monthly rent found in the OCR text."
    },
    "prorated_rent": {
      "type": ["number", "null"],
      "description": "The prorated rent found in the OCR text."
    },
    "security_deposit": {
      "type": ["number", "null"],
      "description": "The security deposit found in the OCR text."
    },
    "lease_rent": {
      "type": ["number", "null"],
      "description": "The security deposit found in the OCR text."
    },
    "monthly_payment_breakdown": {
      "type": ["object", "null"],
      "description": "The monthly payment breakdown data found in the OCR text.",
      "additionalProperties": {}
    },
    "utility_charges": {
      "type": ["object", "null"],
      "description": "The utility charges found in the OCR text. This is a dictionary with utility charges as the key, and their price as the value.",
      "additionalProperties": {
        "type": ["number", "null"]
      }
    }
  },
  "required": ["tenant_name", "unit_address", "unit_number", "unit_type", "agreement_date", "lease_start", "lease_end", "lease_auto_renew", "hourly_rate", "monthly_rent", "prorated_rent", "security_deposit", "lease_rent", "monthly_payment_breakdown", "utility_charges"]
}

extraction_json_schema_str = json.dumps(extraction_json_schema, indent=2)

### Build the Dataset
Build the dataset from the collected data in the Sqlite3 database

In [3]:
import pandas as pd
import sqlite3

# Path to the SQLite database
db_path = "../output/extracted_lease_agreements.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Query to select all data from the extracted_data table
query = "SELECT * FROM extracted_data"

# Read the data into a DataFrame
df = pd.read_sql_query(query, conn, index_col="id")

# Close the database connection
conn.close()

df

,extracted_text,extracted_fields
id,,
1,"Severability, Originals and Attachments, and S...","{""tenant_name"": ""Lazarus Hennesy Michael Debro..."
2,9. FLOOD DISCLOSURE. In compliance with the Ok...,"{""tenant_name"": null, ""unit_address"": null, ""u..."
3,APARTMENT LEASE CONTRACT\nNATIONAL APARTMENT A...,"{""tenant_name"": ""Lazarus Hennesy JR"", ""unit_ad..."
4,FORCE MAJEURE: If we are prevented from comple...,"{""tenant_name"": null, ""unit_address"": null, ""u..."
5,26.CONDITION OF THE PREMISES AND ALTERATIONS. ...,"{""tenant_name"": null, ""unit_address"": null, ""u..."
...,...,...
285,26.CONDITION OF THE PREMISES AND ALTERATIONS. ...,"{""tenant_name"": null, ""unit_address"": null, ""u..."
286,NAA NATIONAL APARTMENT ASSOCIATION We Lead the...,"{""tenant_name"": ""Novy Jezkova"", ""unit_address""..."
287,We :unselected: require :unselected: do not re...,"{""tenant_name"": null, ""unit_address"": null, ""u..."


#### Preprocess the dataset
We need to add in missing fields in the dataset

In [4]:
# Function to ensure all required fields are present in the extracted_fields column
def ensure_all_fields(extracted_fields, required_fields):
    # Convert the extracted_fields string to a dictionary
    extracted_data = json.loads(extracted_fields)
    
    # Check and add any missing fields with a value of None
    for field in required_fields:
        if field not in extracted_data:
            extracted_data[field] = None
    
    # Convert the dictionary back to a JSON string
    return json.dumps(extracted_data)

# List of required fields from the extraction_json_schema
extraction_fields = list(extraction_json_schema['properties'].keys())

# Apply the function to each row in the extracted_fields column
df['extracted_fields'] = df['extracted_fields'].apply(ensure_all_fields, required_fields=extraction_fields)

# Display the updated DataFrame
df

,extracted_text,extracted_fields
id,,
1,"Severability, Originals and Attachments, and S...","{""tenant_name"": ""Lazarus Hennesy Michael Debro..."
2,9. FLOOD DISCLOSURE. In compliance with the Ok...,"{""tenant_name"": null, ""unit_address"": null, ""u..."
3,APARTMENT LEASE CONTRACT\nNATIONAL APARTMENT A...,"{""tenant_name"": ""Lazarus Hennesy JR"", ""unit_ad..."
4,FORCE MAJEURE: If we are prevented from comple...,"{""tenant_name"": null, ""unit_address"": null, ""u..."
5,26.CONDITION OF THE PREMISES AND ALTERATIONS. ...,"{""tenant_name"": null, ""unit_address"": null, ""u..."
...,...,...
285,26.CONDITION OF THE PREMISES AND ALTERATIONS. ...,"{""tenant_name"": null, ""unit_address"": null, ""u..."
286,NAA NATIONAL APARTMENT ASSOCIATION We Lead the...,"{""tenant_name"": ""Novy Jezkova"", ""unit_address""..."
287,We :unselected: require :unselected: do not re...,"{""tenant_name"": null, ""unit_address"": null, ""u..."


### Split the dataset

In [5]:
from sklearn.model_selection import train_test_split

# First split: 80% for training, 20% for temp (which will be split into eval and test)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Second split: 50% of temp for eval and 50% for test (10% of the original data each)
eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Check the sizes of the splits
print(f"Training set size: {len(train_df)}")
print(f"Evaluation set size: {len(eval_df)}")
print(f"Test set size: {len(test_df)}")


Training set size: 231
Evaluation set size: 29
Test set size: 29


### Load the base model

In [6]:
from peft import LoraConfig, TaskType

peft_config = LoraConfig(
    lora_alpha = 16, # increased alpha
    lora_dropout=0.1, # decreased dropout
    r=8, # increased rank
    task_type=TaskType.CAUSAL_LM,
    bias="none",
    target_modules=['k_proj', 'q_proj', 'gate_proj', 'o_proj', 'v_proj', 'down_proj', 'up_proj'], # target as many layers as possible
)

In [7]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import notebook_login
import torch

# login to access gated model
notebook_login()

# Load the tokenizer and model
model_id = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# empty GPU memory
torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


model-00001-of-00002.safetensors:  94%|#########3| 4.65G/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### Build the datasets

In [8]:
from datasets import Dataset

# Create datasets with the extracted text and labels
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset

Dataset({
    features: ['extracted_text', 'extracted_fields', 'id'],
    num_rows: 231
})

### Define the prompt template

In [9]:
prompt_template = """
## USER:
You are a specialized model for extracting specific information from lease agreement text. Your goal is to accurately extract data fields from the provided OCR text of a lease agreement. Additionally, correct any obvious OCR errors you encounter during extraction.

### INPUT TEXT:
Below is the OCR text extracted from a lease agreement. Carefully analyze this text, and extract the relevant data fields.

OCR Text:
```
{extracted_text}
```

### RESPONSE FORMAT:
Return the extracted data as a JSON object, adhering strictly to the following JSON schema:

```json
{extraction_json_schema_str}
```

## ASSISTANT:
```json
{extracted_fields}
```
"""

### Train (fine tune) the base model

In [10]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer

def formatting_func(example):
    output_texts = []
    for i in range(len(example['extracted_text'])):
        text = prompt_template.format(extracted_text=example['extracted_text'][i], extraction_json_schema_str=extraction_json_schema_str, extracted_fields=example['extracted_fields'][i])
        output_texts.append(text)
    return output_texts

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",  # Changed to evaluate based on steps
    eval_steps=10,  # Evaluate every 10 steps
    learning_rate=1e-3,
    lr_scheduler_type="linear",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    logging_steps=2,
    warmup_steps = 2,
    max_steps = 200,
    save_strategy="steps",
    save_steps=10,  # Save a checkpoint every 10 steps
    optim="paged_adamw_32bit",
)

trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    neftune_noise_alpha=5,
    max_seq_length=512,
    args=training_args,
    formatting_func=formatting_func,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,  # No masked language modeling, suitable for CausalLM
        return_tensors="pt"  # Return PyTorch tensors
    )
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/home/azureuser/py_envs/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': neftune_noise_alpha, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)


/home/azureuser/py_envs/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/azureuser/py_envs/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:313: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/231 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


...

/home/azureuser/py_envs/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


/home/azureuser/py_envs/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


/home/azureuser/py_envs/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


/home/azureuser/py_envs/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


TrainOutput(global_step=200, training_loss=0.30718498200178146, metrics={'train_runtime': 13981.27, 'train_samples_per_second': 0.458, 'train_steps_per_second': 0.014, 'total_flos': 3.899302471532544e+16, 'train_loss': 0.30718498200178146, 'epoch': 27.586206896551722})

### Save the model

In [11]:
peft_model_repo_id = "aryaniyaps/finetuned_gemma_2b_lease_data_extraction_40_deals"
local_model_id = "./saved_models/finetuned_gemma_2b_lease_data_extraction_40_deals"

In [12]:
# Push the PEFT model (LoRA adapters) to Hugging Face Hub
trainer.model.save_pretrained(local_model_id)

trainer.model.push_to_hub(peft_model_repo_id)

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aryaniyaps/finetuned_gemma_2b_lease_data_extraction_40_deals/commit/e4c3fa5f76bdfe655e607482a9314001677e1ce3', commit_message='Upload model', commit_description='', oid='e4c3fa5f76bdfe655e607482a9314001677e1ce3', pr_url=None, pr_revision=None, pr_num=None)

### Evaluate the model

In [13]:
from peft import PeftModel
 
# Load the Lora model
finetuned_model = PeftModel.from_pretrained(model, peft_model_repo_id, device_map={"":0})
finetuned_model.eval()
 
print("PEFT model loaded")

adapter_config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

PEFT model loaded


### Evaluate the model
(Using F1 score and exact matches)

In [14]:
evaluation_prompt_template = """
## USER:
You are a specialized model for extracting specific information from lease agreement text. Your goal is to accurately extract data fields from the provided OCR text of a lease agreement. Additionally, correct any obvious OCR errors you encounter during extraction.

### INPUT TEXT:
Below is the OCR text extracted from a lease agreement. Carefully analyze this text, and extract the relevant data fields.

OCR Text:
```
{extracted_text}
```

### RESPONSE FORMAT:
Return the extracted data as a JSON object, adhering strictly to the following JSON schema:

```json
{extraction_json_schema_str}
```

## ASSISTANT:
"""

In [15]:
from peft import PeftModelForCausalLM

finetuned_model = PeftModelForCausalLM.from_pretrained(model=model, model_id=local_model_id)

# Set the model to evaluation mode
finetuned_model.eval()

# Prepare lists to store outputs and true labels
predictions = []
references = []

def format_prompt(message):
    return evaluation_prompt_template.format(extracted_text=message['extracted_text'], extraction_json_schema_str=extraction_json_schema_str)

# Disable gradient calculation for evaluation
with torch.inference_mode():
    for message in test_dataset:
        # Prepare input using apply_chat_template
        input_ids = tokenizer(format_prompt(message), return_tensors="pt").to("cuda")
        # Get the true output
        true_output = message['extracted_fields']
        references.append(true_output)

        # Generate output from the fine-tuned model
        outputs_finetuned = finetuned_model.generate(**input_ids, max_new_tokens=1024, do_sample=False)
        finetuned_output = tokenizer.decode(outputs_finetuned[0])
        predictions.append(finetuned_output)


In [16]:
from tabulate import tabulate

# Prepare data for tabulation
table_data = []
for i, (pred, ref) in enumerate(zip(predictions, references), 1):
    table_data.append([i, ref, pred])

# Define table headers
headers = ["#", "Reference data", "Extracted data (Finetuned model)"]

# Display the table
tabulate(table_data, headers=headers, tablefmt="html", showindex=False)


'<table>\n<thead>\n<tr><th style="text-align: right;">  #</th><th>Reference data                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         </th><th>Extracted data (Finetuned model)  </th></tr>\n</thead>\n<tbody>\n<tr><td style="text-align: right;">  1</td><td>{&quot;tenant_name&quot;: &quot;Dominick DiPrima&quot;, &quot;unit_address&quot;: &quot;245 Eldridge Street, New York, NY 10002&quot;, &quot;unit_number&quot;: &quot;1R&quot;, &quot;unit_type&quot;: &quot;Apartment&quot;, &quot;agreement_date&quot;: &quot;2022-07-27&quot;, &quot;lease_start&quot;: &quot;2022-08-15&quot;, &quot;lease_end&quot;: &quot;2023-07-31&quot;, &quot;lease_auto_renew&quot;: null, &quot;hourly_rate&quot;: null, &quot;monthly_rent&quot;: 8300, &quot;prorated_rent&quot;: null, &quot;security_deposit&quot;: 8300, &quot;lease_rent&quot;: 99600, &quot;monthly_payment_breakdown&quot;: null, &quot;utility_charges&quot;: null}                                                                                                                                                                                           </td><td>&lt;bos&gt;\n## USER:\nYou are a specialized model for extracting specific information from lease agreement text. Your goal is to accurately extract data fields from the provided OCR text of a lease agreement. Additionally, correct any obvious OCR errors you encounter during extraction.\n\n### INPUT TEXT:\nBelow is the OCR text extracted from a lease agreement. Carefully analyze this text, and extract the relevant data fields.\n\nOCR Text:\n```\nEldridge 245 LLC\nSTANDARD FORM OF APARTMENT LEASE (FOR APARTMENTS NOT SUBJECT TO THE RENT STABILIZATION LAW) THE REAL ESTATE BOARD OF NEW YORK, INC. @Copyright 2019. All Rights Reserved. Reproduction in whole or in part prohibited.\nREBNY Apt non-stab 2019 Rev 7.19\nPREAMBLE: This lease contains the agreements between Tenant and Owner concerning the rights and obligations of each party. Tenant and Owner have other rights and obligations which are set forth in government laws and regulations.\nTenant should read this Lease carefully. If Tenant has any questions, or if Tenant does not understand any words or statements herein, obtain clarification from an attorney. Once Tenant and Owner sign this Lease, Tenant and Owner will be presumed to have read it and understood it completely. Tenant and Owner admit that all agreements between Tenant and Owner have been written into this Lease. Tenant understands that any agreements made before or after this Lease was signed and not written into it will not be enforceable.\nTHIS LEASE is made as of July 27, 2022\nbetween Owner(hereinafter referred to as &quot;Owner&quot; or &quot;Lessor&quot;), Eldridge 245 LLC whose address is 215 Lexington Avenue, 20th Floor, New York, NY 10016\nand Tenant (hereinafter referred to &quot;Tenant&quot; or &quot;Lessee&quot; ), Dominick DiPrima, William R. Humphrey, IV, and Maxwell C. Cummings\nwhose address is 245 Eldridge Street #1R, New York, NY 10002.\nPlease note the following paragraphs that require a selection among alternative wording: 2, 3E, 34\nPlease note the following paragraphs that require deletions if inapplicable: 9D, 12C(ii), 12E, 25, 32C(i), 33, 34, 35, 36, 37,38,59,60\nPlease note the following paragraphs that require the insertion of terms (and/or delete if inapplicable): 1, 2, 3A, 3B, 4, 9D, 12B, 12C, 25, 32C, 34A, 35, 38B, Exhibit A (Memorandum Confirming Term), Exhibit C (Owner&#x

In [17]:
# TODO: calculate accuracy using metrics